In [1]:
import requests
import json
import time

api= "45929254f44e4117aa634667a2c345b8"
episode_id = "71de733737a74d4994b0d4d58ebbeafe"
podcast_id = "38a7962c6bfe40318ba5b72772c61741"

upload_endpoint = 'https://api.assemblyai.com/v2/upload'
transcript_endpoint = 'https://api.assemblyai.com/v2/transcript'
#####
listennotes_episode_endpoint = "https://listen-api.listennotes.com/api/v2/episodes"

listennotes_headers = {"X-ListenAPI-Key" : podcast_id}
headers_auth_only = {"authorization" : api}


#headers = {"authorization" : api,
#           "content-type" : "application/json" }

chunk_size = 5242880 # 5mb

def get_episode_auido_url(id):
    url = listennotes_episode_endpoint + "/" + id
    response = requests.request("GET",url,headers = listennotes_headers)
    data = response.json()

    audio_url = data["audio"]
    episode_thumbnail = data["thumbnail"]
    podcast_title = data["podcast"]["title"]
    episode_title = data["title"]

    return audio_url,episode_thumbnail,podcast_title,episode_title

def upload(filename):
    def read_file(filename):
        with open(filename, "rb") as f:
            while True:
                data = f.read(chunk_size)
                if not data:
                    break
                yield data

    upload_response = requests.post(upload_endpoint,headers = headers_auth_only, data = read_file(filename))
    return upload_response.json["upload_url"]

def transcribe(audio_url,auto_chapters):
    transcript_request = {
        "audio_url" : audio_url,
        "auto_chapters" :auto_chapters
    }
    transcript_response = requests.post(transcript_endpoint,headers = headers_auth_only , json = transcript_request)
    return transcript_response.json()["id"]

def poll(transcprit_id):
    polling_endpoint = transcript_endpoint + "/" + transcprit_id
    polling_response = requests.get(polling_endpoint,headers = headers_auth_only)
    return polling_response.json()

def get_transcription_result_url(url,auto_chapters):
    transcribe_id = transcribe(url,auto_chapters)
    while True:
        data = poll(transcribe_id)
        if data["status"] == "completed":
            return data,None
        elif data["status"] == "error":
            return data, data["error"]
    print("Waiting For 5 Seconds...")
    time.sleep(5)

def save_transcript(episode_id):
    audio_url,episode_thumbnail,podcast_title,episode_title = get_episode_auido_url(episode_id)
    data, error = get_transcription_result_url(audio_url,auto_chapters = True) 

    if data:
        filename = episode_id + ".txt"
        with open(filename, "w") as f:
            f.write(data["text"])
        
        chapters_filename = episode_id + "_chapters.json"
        with open(chapters_filename, "w") as f:
            chapters = data["chapters"]

            episode_data = {"chapters": chapters}
            episode_data["episode_thumbnail"] = episode_thumbnail
            episode_data["episode_title"] = episode_title
            episode_data["podcast_title"] = podcast_title

            json.dump(episode_data,f)
            print("Transcript Saved ...")
            return True

    elif error:
        print("Error : " ,error)
        return False

In [2]:
get_episode_auido_url(episode_id)

('https://www.listennotes.com/e/p/71de733737a74d4994b0d4d58ebbeafe/',
 'https://cdn-images-1.listennotes.com/podcasts/99-invisible-roman-mars-cujtwfGCesU-JqxekyhmQi1.300x300.jpg',
 '99% Invisible',
 '491- The Missing Middle')

In [3]:
save_transcript(episode_id)

Transcript Saved ...


True

In [ ]:
import streamlit as st
import json

st.title("Welcome to my application that creates podcast summaries")
episode_id = st.sidebar.text_input("Please input an episode id")
button = st.sidebar.button("Get podcast summary!",on_click = save_transcript,args = (episode_id, ))

def get_clean_time(start_ms):
    seconds = int((start_ms/1000)%60)
    minutes = int((start_ms/(100*60)) % 60)
    hours = int((start_ms/(100*60)) % 24)
    if hours > 0:
        start_t = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
    else:
        start_t = f"{minutes:02d}:{seconds:02d}"
    return start_t

if button:

    filename = episode_id + "_chapters.json"
    with open(filename,"r") as f:
        data = json.load(f)
        chapters = data["chapters"]
        podcast_title = data["podcast_title"]
        episode_title = data["episode_title"]
        thumbnail = data["episode_thumbnail"]

    st.header(f"{podcast_title} - {episode_title}")
    st.image(thumbnail)
    for chp in chapters:
        with st.expander(chp["gist"] + "-" + get_clean_time(chp["start"])):
            chp["summary"]
